In [96]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import json
from time import sleep
import requests_cache
from functools import reduce
from requests_cache import NEVER_EXPIRE

In [97]:
def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')

    data = []

    for table in tables:
        category = table.find('h3').text.strip()
        rows = table.find_all('tr')[1:]  # Skip the title row

        for row in rows:
            feature = row.find('th').text.strip()
            value = row.find('td').text.strip()
            if not value and 'class' in row.find('td').attrs:
                value = 'Yes' if 'yes' in row.find('td')['class'] else 'No'
            data.append({'Category': category, 'Feature': feature, 'Value': value})

    return pd.DataFrame(data)

In [98]:
def request_function(url, agent_list, session):
    attempts = 5

    while attempts > 0:
        random_agent_row = agent_list.sample()
        random_agent = random_agent_row['Agent'].item()
        response = session.get(url, headers={'User-Agent': random_agent})

        content = response.text
        if '<title>Garmin | Page not found</title>' in content:
            print(attempts, 'Unused Page')
            return 'Error - Unused Page', pd.DataFrame()
        elif 'Access Denied' in content:
            print(attempts, 'Access Denied')
            session.cache.delete_url(url)
            print(attempts, 'Access Denied')
        elif 'var GarminAppBootstrap' in content:
            return 'Data Found', content
        attempts -= 1
        sleep(5)
    return 'Error - attempts ran out', pd.DataFrame()


In [99]:
def parse_spec_page(content, url):
    soup = BeautifulSoup(content, 'html.parser')

    garmin_app_bootstrap = soup.find('div', id='garmin-app-bootstrap')
    try:
        g_scipt = garmin_app_bootstrap.find('script')
    except Exception as e:
        print(content)
        print(e)
        raise AttributeError

    match = re.search(r'var GarminAppBootstrap = ({.*?});\s*\n', g_scipt.text, re.DOTALL)

    dict_data = match.group(1)

    data = json.loads(dict_data)

    main_sku = data['sku']
    skus = data['skus']

    seoAttributes = data['seoAttributes']
    # print(seoAttributes)

    product_attributes = skus[main_sku]

    productName = product_attributes['productName']
    productVariation = product_attributes['productVariation']
    productId = product_attributes['productId']
    try:
        price = product_attributes['price']['price']['price']
    except Exception as e:
        price = ''
    image_link = product_attributes['seo']['ogImage']

    specs = product_attributes['tabs']['specsTab']['content']
    spec_df = parse_html(specs)
    # Add product name and variation to the spec_df as new rows with a Category of 'Product Info' using concat
    # make a small dataframe with the product name and variation
    df_9 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Product Name - no URL', 'Value': productName},
                        index=[0])
    df_1 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Product Name',
                         'Value'   : f"""HYPERLINK("{url}", "{productName}")"""}, index=[0])
    df_2 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Product Variation', 'Value': productVariation},
                        index=[0])
    df_3 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Product ID', 'Value': productId}, index=[0])
    df_4 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Main SKU', 'Value': main_sku}, index=[0])
    # df_5 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Image Link', 'Value': image_link}, index=[0])
    df_8 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Image', 'Value': f"""IMAGE("{image_link}")"""},
                        index=[0])
    df_6 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Product URL', 'Value': url}, index=[0])
    df_7 = pd.DataFrame({'Category': 'Product Info', 'Feature': 'Price', 'Value': price}, index=[0])

    spec_df = pd.concat([df_1, df_2, df_3, df_4, df_6, df_7, df_8, df_9, spec_df], ignore_index=True)
    spec_df = spec_df.rename(columns={'Value': productName})
    return productName, spec_df


In [100]:
def get_live_data(id, session, agent_list, gb_list=[], au_list=[], debug=False):
    if id in gb_list:
        url = f'https://www.garmin.com/en-GB/p/{id}'
    elif id in au_list:
        url = f'https://www.garmin.com/en-AU/p/{id}'
    else:
        url = f'https://www.garmin.com/en-US/p/{id}'
    if debug: print(url)

    err_text, content = request_function(url, agent_list, session)
    if 'Error - attempts ran out' in err_text:
        print(err_text)
        raise Exception
    elif 'Error - Unused Page' in err_text:
        return 'Unused Page', pd.DataFrame()

    productName, spec_df = parse_spec_page(content, url)
    return productName, spec_df

In [101]:
ids = [662694, 662825, 742133, 785411, 665374, 667002, 667005, 672620, 270, 27335, 595326, 104012, 10527, 1055469,
       1057989, 107143, 107272, 108731, 10884, 10885, 11039, 112885, 112912, 1196129, 120680, 122785, 1228171,
       1228429, 1228493, 126596, 1277573, 129397, 1297277, 1300797, 1323517, 134491, 134596, 137024, 138810,
       1390829, 139389, 1398869, 140528, 1411809, 143405, 143677, 145621, 148289, '150767/pn/010-01297-01', 154886,
       159116, 160512, 1611937, 164366, 1652337, 166370, 230, 231, 27483, 31859, 331, 348, 349, 36728, 504038, 508487,
       508489, 512475, 512478, 516105, 516207, 519298, 522791, 523893, 527935, 529988, 531166, 532035, 532348, 538374,
       539963, 541225, 543199, 543589, 544984, 545411, 545480, 548743, 552237, 552962, 552982, 553168, 554662, 560327,
       561299, 564291, 567813, 567991, 568181, 571520, 574602, 580642, 582444, 583562, 583980, 591311, 591945, 596828,
       597253, 602063, 602068, 603201, 603229, 603267, 605172, 605739, 608567, 608613, 608618, 608665, 611037,
       611996, 612476, 621224, 621232, 621802, 621922, 623539, 623921, 628939, 632320, 633356, 633625, 633646, 633654,
       633685, 63511, 635923, 6400, 641121, 641375, 641435, 641449, 641479, 641501, 641530, 643260, 643382, 643399,
       646690, 647267, 658892, 662790, 679335, 682416, 690051, 69043, 695391, 695447, 695448, 696004, 696005, 698394,
       698436, 698519, 698632, 699548, 699976, 700437, 701618, 702797, 704417, 707174, 707225, 707506, 707538, 707572,
       711488, 713363, 714945, 715736, 715870, 716891, 721216, 725447, 730659, 731136, 733535, 739416, 741137, 741374,
       741438, 741462, 742152, 742170, 742174, 742207, 743387, 760778, 761998, 766516, 766565, 775357, 775421, 775697,
       775739, 777655, 777730, 780139, 780165, 780196, 782585, 783441, 783448, 783461, 783465, 783467, 798161, 798777,
       798925, 798926, 798938, 798946, 801643, 802162, 802703, 802901, 802925, 815409, 815448, 815582, 818345, 818387,
       819546, 819727, 819761, 83068, 83274, 83280, 84374, 847697, 847706, 851039, 852159, 852183, 852217, 854515,
       869433, 873008, 873214, 884088, 884198, 884292, 884414, 884585, 886689, 886725, 886785, 894039, 894067, 905283,
       90671, 90675, 950157, 950277, 97287, 681612, 735611, 865822, 1479414, 735563, 1132094, 770963, 649059, 512311,
       530376, 136403, 719696, 711538, 530464, 679362, 658593, 699171, 658594, 658586, 699230, 658661, 573589,
       606555, 518151, 721258, 698001, 669024, 550821, 601468, 885302, 701670, 734868, 690885, 874099,
       690887, 690888, 690890, 690886, 690885, 742197, 815332, 866230, 1479413, 735520, 866139, 865925,
       735542, 866191, 865945, 682155, 260, 257, 30026, 30025, 572639, 583825, 858933, 859036, 681757, 637006, 711489]

gb_list = [573589, 707538, 761998, 532348, 612476, 633356, 690051, 707225, 707538, 715736, 761998, 633356]

au_list = [530376, 136403, 682155]

# vectors = [510941, 167943, 93569] - Don't have the same kind of spec layout

# 600380 - a vivofit Jr 2 First Order that is not saved anywehre in English
# 696564 - A Tacx Genius Trainer, too old to have a normal page
# 696562 - A Tacx Bushido Trainer, too old to have a normal page
# 697357 - A Tacx Neo Smart Trainer, too old to have a normal page
# 696563 - A Tacx FLUX Smart Trainer, too old to have a normal page
# 305658 - HRM PRo for the US, doesn't work anymore, 682155 does fur en-AU

# initialize the dataframe with columns Category, Feature, and Value
df = pd.DataFrame(columns=['Category', 'Feature'])

agent_list = pd.read_excel('agent_list.xlsx', engine='openpyxl')

debug = True

session = requests_cache.CachedSession('garmin_cache')
session.settings.expire_after = NEVER_EXPIRE

dfs = []

for id in ids:
    productName, id_df = get_live_data(id, session, agent_list, gb_list, au_list, debug)
    # join the dataframes together based on matching the Category and Feature columns
    if debug: print(id, productName)
    # check if productName is not one of the error messages
    if productName not in ['Unused Page', 'Access Denied']:
        dfs.append(id_df)

# iterate over all files in the saved_htmls folder
for file in os.listdir('saved_htmls'):
    with open(f'saved_htmls/{file}', 'r', encoding='utf-8') as f:
        content = f.read()
    file_name = file.split('.')[0]
    productName, id_df = parse_spec_page(content, f'https://www.garmin.com/en-US/p/{file_name}')
    if debug: print(file, productName)
    dfs.append(id_df)

# merge the dataframes together based on matching the Category and Feature columns

df = reduce(lambda left, right: pd.merge(left, right, on=['Category', 'Feature'], how='outer'), dfs)

# sort all the columns in the dataframe except for the first two columns
first_two = df.columns[:2].tolist()
sorted_rest = sorted(df.columns[2:])
new_order = first_two + sorted_rest
df = df.reindex(columns=new_order)

https://www.garmin.com/en-US/p/662694
662694 vívomove® Luxe
https://www.garmin.com/en-US/p/662825
662825 vívomove® Style
https://www.garmin.com/en-US/p/742133
742133 vívomove® Sport
https://www.garmin.com/en-US/p/785411
785411 vívomove® Trend
https://www.garmin.com/en-US/p/665374
665374 Garmin Swim™ 2
https://www.garmin.com/en-US/p/667002
667002 Rey™
https://www.garmin.com/en-US/p/667005
667005 Captain Marvel
https://www.garmin.com/en-US/p/672620
672620 Approach® Z82
https://www.garmin.com/en-US/p/270
270 Forerunner® 301
https://www.garmin.com/en-US/p/27335
27335 Forerunner® 310XT
https://www.garmin.com/en-US/p/595326
595326 quatix® 5 Sapphire
https://www.garmin.com/en-US/p/104012
104012 FR70
https://www.garmin.com/en-US/p/10527
10527 Forerunner® 50
https://www.garmin.com/en-US/p/1055469
1055469 Forerunner® 165
https://www.garmin.com/en-US/p/1057989
1057989 vívoactive® 5
https://www.garmin.com/en-US/p/107143
107143 Forerunner® 10
https://www.garmin.com/en-US/p/107272
107272 fēnix®
http

In [102]:
df.to_excel('garmin_specs.xlsx', index=False)

In [103]:
df = pd.read_excel('garmin_specs.xlsx', engine='openpyxl')

In [36]:
# TODO: create new rows for the following ['Battery Life: Smartwatch', 'Battery Life: GPS', 'Battery Life: Max Battery GPS', 'Battery Life: Expedition GPS', 'Battery Life: Pool and OHR mode', 'Battery Life: Battery Saver Watch Mode', 'Battery Life: All Satellite Systems', 'Battery Life: All Satellite Systems and Multi-band', 'Battery Life: All Satellite Systems and Music', 'Battery Life: GPS + Music', 'Battery Life: UltraTrac', 'Battery Life: Dive mode', 'Battery Life: All-Systems GNSS mode']

In [107]:
df_temp = df.copy()

# keep only the first 10 columns of the dataframe
# df_temp = df_temp.iloc[:, :10]


category_feature_sort = pd.read_excel('CategoryFeatureSort.xlsx', engine='openpyxl')
# Join the dataframes together based on matching the Category and Feature columns, replacing the index column with the "ConcatVal" col of the category_feature_sort dataframe
df_temp = df_temp.merge(category_feature_sort, how='outer', on=['Category', 'Feature'])
df_temp = df_temp.sort_values(by='ConcatVal')
# drop the CatVal, FeatVal, ConcatVal, Concat columns
df_temp = df_temp.drop(columns=['CatVal', 'FeatVal', 'ConcatVal', 'Concat'])

df_temp.head(10)

,Category,Feature,fēnix® 3 HR,Approach® CT10 - Full Set,Approach® CT10 - Starter Pack,Approach® G12,Approach® G30,Approach® G80,Approach® R10,Approach® S1,...,vívomove® Sport,vívomove® Style,vívomove® Trend,vívosmart®,vívosmart® 3,vívosmart® 4,vívosmart® 5,vívosmart® HR,vívosmart® HR+,vívosport®
664,Product Info,Product Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,Product Info,Product Name,"HYPERLINK(""https://www.garmin.com/en-US/p/5454...","HYPERLINK(""https://www.garmin.com/en-US/p/6051...","HYPERLINK(""https://www.garmin.com/en-US/p/7981...","HYPERLINK(""https://www.garmin.com/en-US/p/7394...","HYPERLINK(""https://www.garmin.com/en-US/p/5531...","HYPERLINK(""https://www.garmin.com/en-US/p/5972...","HYPERLINK(""https://www.garmin.com/en-US/p/6953...","HYPERLINK(""https://www.garmin.com/en-US/p/8306...",...,"HYPERLINK(""https://www.garmin.com/en-US/p/7421...","HYPERLINK(""https://www.garmin.com/en-US/p/6628...","HYPERLINK(""https://www.garmin.com/en-US/p/7854...","HYPERLINK(""https://www.garmin.com/en-US/p/1548...","HYPERLINK(""https://www.garmin.com/en-US/p/5678...","HYPERLINK(""https://www.garmin.com/en-US/p/6057...","HYPERLINK(""https://www.garmin.com/en-US/p/7825...","HYPERLINK(""https://www.garmin.com/en-US/p/5311...","HYPERLINK(""https://www.garmin.com/en-US/p/5487...","HYPERLINK(""https://www.garmin.com/en-US/p/5746..."
668,Product Info,Product Variation,Slate Gray with Black Band,Full Set,Starter Pack,Device Only,Approach® G30,Approach® G80,Portable Golf Launch Monitor,"Black, U.S.",...,Black Case and Silicone Band with Slate Accents,Graphite Aluminum Case with Black Pepper Woven...,Cream Gold Stainless Steel Bezel with French G...,Berry (Small),Blue with Stainless Buckle (Small/Medium),Black with Midnight Hardware,Black,Midnight Blue (Regular),Black/Shark Fin Gray,Fuchsia Focus (Small/Medium)
659,Product Info,Image,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...",...,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-..."
661,Product Info,Main SKU,010-01338-70,010-01994-00,010-01994-01,010-02555-00,010-01690-00,010-01914-00,010-02356-00,010-00932-06,...,010-02566-00,010-02240-03,010-02665-02,010-01317-03,010-01755-12,010-01995-10,010-02645-00,010-01955-08,010-01955-36,010-01789-11
663,Product Info,Product ID,545480,605172,798161,739416,553168,597253,695391,83068,...,742133,662825,785411,154886,567813,605739,782585,531166,548743,574602
662,Product Info,Price,NaN,299.99,79.99,149.99,249.99,499.99,599.99,NaN,...,179.99,NaN,299.99,NaN,NaN,NaN,149.99,NaN,NaN,NaN
667,Product Info,Product URL,https://www.garmin.com/en-US/p/545480,https://www.garmin.com/en-US/p/605172,https://www.garmin.com/en-US/p/798161,https://www.garmin.com/en-US/p/739416,https://www.garmin.com/en-US/p/553168,https://www.garmin.com/en-US/p/597253,https://www.garmin.com/en-US/p/695391,https://www.garmin.com/en-US/p/83068,...,https://www.garmin.com/en-US/p/742133,https://www.garmin.com/en-US/p/662825,https://www.garmin.com/en-US/p/785411,https://www.garmin.com/en-US/p/154886,https://www.garmin.com/en-US/p/567813,https://www.garmin.com/en-US/p/605739,https://www.garmin.com/en-US/p/782

In [108]:
# load the spec_extras.xlsx file into a dataframe
df_extras = pd.read_excel('spec_extras.xlsx', engine='openpyxl')

# get the index of the row containing "Release Date"
release_date_index = df_temp[df_temp['Feature'] == 'Release Date'].index[0]
review_index = df_temp[df_temp['Feature'] == 'DCRainmaker Review'].index[0]
product_id_index = df_temp[df_temp['Feature'] == 'Product ID'].index[0]

# iterate over each column in the dataframe except for the first two columns
for column in df_temp.columns[2:]:
    # get the value in the column for the product_id_index
    product_id = df_temp[column][product_id_index]
    product_info = df_extras[df_extras['Product ID'] == int(product_id)]

    if not product_info.empty:
        # Extract the Review and Release Date values
        review = product_info['DCRainmaker Review'].values[0]
        release_date = product_info['Release Date'].values[0]
        # set the values in the dataframe with loc
        df_temp.loc[review_index, column] = review
        df_temp.loc[release_date_index, column] = release_date

df_temp.head(10)


,Category,Feature,fēnix® 3 HR,Approach® CT10 - Full Set,Approach® CT10 - Starter Pack,Approach® G12,Approach® G30,Approach® G80,Approach® R10,Approach® S1,...,vívomove® Sport,vívomove® Style,vívomove® Trend,vívosmart®,vívosmart® 3,vívosmart® 4,vívosmart® 5,vívosmart® HR,vívosmart® HR+,vívosport®
664,Product Info,Product Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,Product Info,Product Name,"HYPERLINK(""https://www.garmin.com/en-US/p/5454...","HYPERLINK(""https://www.garmin.com/en-US/p/6051...","HYPERLINK(""https://www.garmin.com/en-US/p/7981...","HYPERLINK(""https://www.garmin.com/en-US/p/7394...","HYPERLINK(""https://www.garmin.com/en-US/p/5531...","HYPERLINK(""https://www.garmin.com/en-US/p/5972...","HYPERLINK(""https://www.garmin.com/en-US/p/6953...","HYPERLINK(""https://www.garmin.com/en-US/p/8306...",...,"HYPERLINK(""https://www.garmin.com/en-US/p/7421...","HYPERLINK(""https://www.garmin.com/en-US/p/6628...","HYPERLINK(""https://www.garmin.com/en-US/p/7854...","HYPERLINK(""https://www.garmin.com/en-US/p/1548...","HYPERLINK(""https://www.garmin.com/en-US/p/5678...","HYPERLINK(""https://www.garmin.com/en-US/p/6057...","HYPERLINK(""https://www.garmin.com/en-US/p/7825...","HYPERLINK(""https://www.garmin.com/en-US/p/5311...","HYPERLINK(""https://www.garmin.com/en-US/p/5487...","HYPERLINK(""https://www.garmin.com/en-US/p/5746..."
668,Product Info,Product Variation,Slate Gray with Black Band,Full Set,Starter Pack,Device Only,Approach® G30,Approach® G80,Portable Golf Launch Monitor,"Black, U.S.",...,Black Case and Silicone Band with Slate Accents,Graphite Aluminum Case with Black Pepper Woven...,Cream Gold Stainless Steel Bezel with French G...,Berry (Small),Blue with Stainless Buckle (Small/Medium),Black with Midnight Hardware,Black,Midnight Blue (Regular),Black/Shark Fin Gray,Fuchsia Focus (Small/Medium)
659,Product Info,Image,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...",...,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-..."
661,Product Info,Main SKU,010-01338-70,010-01994-00,010-01994-01,010-02555-00,010-01690-00,010-01914-00,010-02356-00,010-00932-06,...,010-02566-00,010-02240-03,010-02665-02,010-01317-03,010-01755-12,010-01995-10,010-02645-00,010-01955-08,010-01955-36,010-01789-11
663,Product Info,Product ID,545480,605172,798161,739416,553168,597253,695391,83068,...,742133,662825,785411,154886,567813,605739,782585,531166,548743,574602
662,Product Info,Price,NaN,299.99,79.99,149.99,249.99,499.99,599.99,NaN,...,179.99,NaN,299.99,NaN,NaN,NaN,149.99,NaN,NaN,NaN
667,Product Info,Product URL,https://www.garmin.com/en-US/p/545480,https://www.garmin.com/en-US/p/605172,https://www.garmin.com/en-US/p/798161,https://www.garmin.com/en-US/p/739416,https://www.garmin.com/en-US/p/553168,https://www.garmin.com/en-US/p/597253,https://www.garmin.com/en-US/p/695391,https://www.garmin.com/en-US/p/83068,...,https://www.garmin.com/en-US/p/742133,https://www.garmin.com/en-US/p/662825,https://www.garmin.com/en-US/p/785411,https://www.garmin.com/en-US/p/154886,https://www.garmin.com/en-US/p/567813,https://www.garmin.com/en-US/p/605739,https://www.garmin.com/en-US/p/782

In [111]:
# Transpose the dataframe
# copy andm then remove the first two columns of the dataframe
df_tempt = df_temp.copy()
release_date_index = df_tempt[df_tempt['Feature'] == 'Release Date'].index[0]
df_tempt = df_tempt.drop(columns=['Category', 'Feature'])

# get the index of the row containing "Release Date"

df_t = df_tempt.T
# sort df_t by column = 667
df_t = df_t.sort_values(by=[release_date_index], na_position='last', ascending=False)
# Transpose the dataframe back
df_tempt = df_t.T
# add the first two columns back to the dataframe
df_tempt = pd.concat([df_temp[['Category', 'Feature']], df_tempt], ignore_index=True, axis=1)
df_tempt.head(10)

,0,1,2,3,4,5,6,7,8,9,...,320,321,322,323,324,325,326,327,328,329
664,Product Info,Product Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,Product Info,Product Name,"HYPERLINK(""https://www.garmin.com/en-US/p/1228...","HYPERLINK(""https://www.garmin.com/en-US/p/8510...","HYPERLINK(""https://www.garmin.com/en-US/p/1228...","HYPERLINK(""https://www.garmin.com/en-US/p/1228...","HYPERLINK(""https://www.garmin.com/en-US/p/1390...","HYPERLINK(""https://www.garmin.com/en-US/p/1411...","HYPERLINK(""https://www.garmin.com/en-US/p/1196...","HYPERLINK(""https://www.garmin.com/en-US/p/1055...",...,"HYPERLINK(""https://www.garmin.com/en-US/p/7114...","HYPERLINK(""https://www.garmin.com/en-GB/p/5323...","HYPERLINK(""https://www.garmin.com/en-US/p/5835...","HYPERLINK(""https://www.garmin.com/en-US/p/7421...","HYPERLINK(""https://www.garmin.com/en-US/p/1548...","HYPERLINK(""https://www.garmin.com/en-US/p/5678...","HYPERLINK(""https://www.garmin.com/en-US/p/6057...","HYPERLINK(""https://www.garmin.com/en-US/p/5311...","HYPERLINK(""https://www.garmin.com/en-US/p/5487...","HYPERLINK(""https://www.garmin.com/en-US/p/5746..."
668,Product Info,Product Variation,"Sapphire, Titanium with Spark Orange/Graphite ...",Carbon Gray DLC Titanium with Black Ultrafit N...,"Sapphire, Soft Gold with Fog Gray/Dark Sandsto...","Sapphire, Titanium with Spark Orange/Graphite ...","Sapphire, Titanium with Amp Yellow/Graphite Si...",Golf Laser Range Finder,Premium Cycling Computer,Black/Slate Gray,...,Black Panther Special Edition,"Classic, White with Leather Band",Silver Stainless Steel Case with Dark Brown Em...,Black Case and Silicone Band with Slate Accents,Berry (Small),Blue with Stainless Buckle (Small/Medium),Black with Midnight Hardware,Midnight Blue (Regular),Black/Shark Fin Gray,Fuchsia Focus (Small/Medium)
659,Product Info,Image,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...",...,"IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-...","IMAGE(""https://res.garmin.com/en/products/010-..."
661,Product Info,Main SKU,010-02904-10,010-02751-00,010-02903-10,010-02905-10,010-02907-20,010-02950-00,010-02890-00,010-02863-20,...,010-02441-34,010-01597-11,010-01850-AD,010-02566-00,010-01317-03,010-01755-12,010-01995-10,010-01955-08,010-01955-36,010-01789-11
663,Product Info,Product ID,1228429,851039,1228493,1228171,1390829,1411809,1196129,1055469,...,711488,532348,583562,742133,154886,567813,605739,531166,548743,574602
662,Product Info,Price,1099.99,899.99,1099.99,1199.99,1199.99,399.99,699.99,249.99,...,89.99,179.99,NaN,179.99,NaN,NaN,NaN,NaN,NaN,NaN
667,Product Info,Product URL,https://www.garmin.com/en-US/p/1228429,https://www.garmin.com/en-US/p/851039,https://www.garmin.com/en-US/p/1228493,https://www.garmin.com/en-US/p/1228171,https://www.garmin.com/en-US/p/1390829,https://www.garmin.com/en-US/p/1411809,https://www.garmin.com/en-US/p/1196129,https://www.garmin.com/en-US/p/1055469,...,https://www.garmin.com/en-US/p/711488,https://www.garmin.com/en-GB/p/532348,https://www.garmin.com/en-US/p/583562,https://www.garmin.com/en-US/p/742133,https://www.garmin.com/en-US/p/154886,https://www.garmin.com/en-US/p/567813,https://www.garmin.com/en-US/p/605739,h

In [113]:
df_tempt.to_csv('test.csv', encoding='utf-8-sig', index=False)